In [2]:
import torch
import torcheeg

In [3]:
import os
# List files in the specified directory
directory_path = '/scratch/connectome/dyhan316/ECOG_PILOT/data_rearranged'
files = os.listdir(directory_path)
print(files)


['final_data_all_flattened.npy', 'all_mashed', 'subject-aggregated', 'final_label_all_flattened.npy', 'final_label_array.npy', 'final_data_array.npy', 'subject-wise']


In [4]:
import numpy as np

# Load the .npy file
data = np.load(os.path.join(directory_path, 'final_data_array.npy'))
data_flattened = np.load(os.path.join(directory_path, 'final_data_all_flattened.npy'))
label = np.load(os.path.join(directory_path, 'final_label_array.npy'))
label_flattened = np.load(os.path.join(directory_path, 'final_label_all_flattened.npy'))

# Check the dimensions of the loaded data
print(data.shape) #3 Subjects X 6 Session X 20 Trials X 1 Channel X 1000 Timepoints
print(data_flattened.shape) #() X 10000 Timepoints
print(label.shape) #3 Subjects X 6 Session X 20 Trials
print(label_flattened.shape)

(3, 6, 20, 1, 10000)
(360, 10000)
(3, 6, 20)
(360,)


In [5]:
data_expand = data_flattened.reshape(1, 1, 360, 10000) #batchsize, num_electrodes, chunk_size
data_expand.shape

(1, 1, 360, 10000)

In [5]:
#import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
#from torchvision import transforms, utils
import pandas as pd

class MyDataset(Dataset):
    def __init__(self, root_dir, data, label, transform=None):
        self.root_dir = root_dir
        self.data = np.load(os.path.join(root_dir, data))
        self.label = np.load(os.path.join(root_dir, label))
        self.transform = transform
        
    def init_io(self):
        self.info = pd.DataFrame({
                "subject_id": [1],
                "trial_id": [1],
                "duration": [10000],
                "_record_id": ["_record_0"]
            })
        
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample_data = torch.tensor(self.data[idx], dtype=torch.float32)  # Ensure data is float32
        sample_label = torch.tensor(self.label[idx], dtype=torch.float32)  # Ensure label is float32
        
        if self.transform:
            sample_data = self.transform(sample_data)
        
        return sample_data, sample_label



In [6]:
#from torch import float32
from torchvision import transforms

transform = transforms.Compose([
    transforms.Lambda(lambda x: x.to(torch.float32))  # Ensure data is float32
])

dataset = MyDataset(directory_path, 'final_data_all_flattened.npy', 'final_label_all_flattened.npy', transform=transform)
dataset.init_io()

/scratch/connectome/ahhyun724/.conda/envs/torcheeg/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/scratch/connectome/ahhyun724/.conda/envs/torcheeg/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [7]:
data_tensor, label_tensor = dataset[0]
print(f"Data tensor type: {data_tensor.dtype}") #should be float32 
print(f"Label tensor type: {label_tensor.dtype}")

Data tensor type: torch.float32
Label tensor type: torch.float32


In [11]:
dataset.info

,subject_id,trial_id,duration,_record_id
0,1,1,10000,_record_0


In [12]:
len(dataset)

360

In [13]:
dataset[0]

(tensor([-96.7910, -95.8556, -95.0470,  ..., -85.6072, -85.7989, -86.1912]),
 tensor(1.))

# Train_test_split

In [90]:
# Not working for torcheeg.model_selection
from torcheeg.model_selection import train_test_split
from torcheeg import transforms
from torch.utils.data import DataLoader

# dataset = DEAPDataset(root_path='./data_preprocessed_python',
#                       online_transform=transforms.Compose([
#                           transforms.To2d(),
#                           transforms.ToTensor()
#                       ]),
#                       label_transform=transforms.Compose([
#                           transforms.Select(['valence', 'arousal']),
#                           transforms.Binary(5.0),
#                           transforms.BinariesToCategory()
#                       ]))

train_dataset, test_dataset = train_test_split(dataset=dataset)

train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)


[2025-01-21 09:10:30] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-01-21 09:10:30] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1737450630557_Lq4JH for the next run, if you want to use the same setting for the experiment.


IndexError: positional indexers are out-of-bounds

# random_split (In Torch library)

In [14]:
## CHUNNKING 

from torch.utils.data import Dataset, DataLoader
import torch

# Custom Dataset for Chunking
class ChunkedDataset(Dataset):
    def __init__(self, data, labels, chunk_size):
        self.data = data
        self.labels = labels
        self.chunk_size = chunk_size
        self.num_chunks = data.shape[1] // chunk_size  # Number of chunks per sample

    def __len__(self):
        return len(self.data) * self.num_chunks

    def __getitem__(self, idx):
        sample_idx = idx // self.num_chunks  # Which sample
        chunk_idx = idx % self.num_chunks   # Which chunk within the sample
        start = chunk_idx * self.chunk_size
        end = start + self.chunk_size

        # Extract the chunk and corresponding label
        chunk = self.data[sample_idx, start:end]
        label = self.labels[sample_idx]
        return chunk, label

# Chunk size
chunk_size = 128

# Create chunked dataset
chunked_dataset = ChunkedDataset(data_flattened, label_flattened, chunk_size)
print(data_flattened.shape)
print(label_flattened.shape)
print(len(chunked_dataset))

# Create DataLoader
def collate_fn(batch):
    data, label = zip(*batch)
    data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
    label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])
    return data, label

train_loader = DataLoader(chunked_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Check a batch
for inputs, targets in train_loader:
    print(f"Inputs shape: {inputs.shape}")  # [batch_size, 1, chunk_size]
    print(f"Targets shape: {targets.shape}")
    break


(360, 10000)
(360,)
28080
Inputs shape: torch.Size([32, 1, 128])
Targets shape: torch.Size([32])


In [8]:
# Using random split in torch
from torch.utils.data import random_split, DataLoader

# [batch_size, 1, timepoint]
def collate_fn(batch):
    data, label = zip(*batch)
    data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
    label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])
    return data, label

# Assuming `dataset` is your full dataset
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)  # 80% for training
test_size = dataset_size - train_size

# Split the dataset into train and test
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [9]:
#checking the first batch size
for batch in train_loader:
    inputs, targets = batch
    print(f"Inputs shape: {inputs.shape}")
    print(f"Targets shape: {targets.shape}")
    break  # Inspect only the first batch

Inputs shape: torch.Size([32, 1, 10000])
Targets shape: torch.Size([32])


/tmp/ipykernel_160839/4218094284.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_160839/4218094284.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


In [9]:
#LSTM model
from torcheeg.models import LSTM

model = LSTM(num_electrodes=1, hid_channels=64, num_classes=2)

x, y = next(iter(train_loader))
output = model(x)

/tmp/ipykernel_163597/4218094284.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_163597/4218094284.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


In [10]:
import torch
from torch.nn.functional import softmax

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No gradients needed for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.float(), labels.long()  # Ensure proper types
            outputs = model(inputs)  # Forward pass
            predictions = torch.argmax(outputs, dim=1)  # Get predicted class
            correct += (predictions == labels).sum().item()  # Count correct predictions
            total += labels.size(0)  # Total samples

    accuracy = correct / total
    return accuracy

# Calculate accuracy
accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.2%}")


/tmp/ipykernel_163597/4218094284.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_163597/4218094284.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


Test Accuracy: 50.00%


In [135]:
from sklearn.metrics import classification_report

# Function to evaluate with detailed metrics
def evaluate_with_metrics(model, test_loader):
    model.eval()  # Set model to evaluation mode
    all_predictions = []
    all_labels = []

    with torch.no_grad():  # No gradients needed for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.float(), labels.long()  # Ensure proper types
            outputs = model(inputs)  # Forward pass
            predictions = torch.argmax(outputs, dim=1)  # Get predicted class
            all_predictions.extend(predictions.cpu().numpy())  # Store predictions
            all_labels.extend(labels.cpu().numpy())  # Store true labels

    # Use sklearn to calculate metrics
    report = classification_report(all_labels, all_predictions, target_names=['Class 0', 'Class 1'])
    print(report)

# Generate and print classification report
evaluate_with_metrics(model, test_loader)


/tmp/ipykernel_45386/4218094284.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_45386/4218094284.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


              precision    recall  f1-score   support

     Class 0       0.44      0.59      0.51        34
     Class 1       0.48      0.34      0.40        38

    accuracy                           0.46        72
   macro avg       0.46      0.47      0.45        72
weighted avg       0.46      0.46      0.45        72



In [19]:
# GRU
from torcheeg.models import GRU

model = GRU(num_electrodes=1, hid_channels=64, num_classes=2)

x, y = next(iter(train_loader))
output = model(x)

/tmp/ipykernel_157287/4218094284.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_157287/4218094284.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


In [20]:
import torch
from torch.nn.functional import softmax

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No gradients needed for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.float(), labels.long()  # Ensure proper types
            outputs = model(inputs)  # Forward pass
            predictions = torch.argmax(outputs, dim=1)  # Get predicted class
            correct += (predictions == labels).sum().item()  # Count correct predictions
            total += labels.size(0)  # Total samples

    accuracy = correct / total
    return accuracy

# Calculate accuracy
accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.2%}")



/tmp/ipykernel_157287/4218094284.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_157287/4218094284.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


Test Accuracy: 52.78%


In [15]:
# Using random split in torch
from torch.utils.data import random_split, DataLoader

# [batch_size, 1, timepoint]
def collate_fn(batch):
    data, label = zip(*batch)
    data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
    data = data.unsqueeze(1)
    data = data[:,:,:,::4]
    label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])
    return data, label

# Assuming `dataset` is your full dataset
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)  # 80% for training
test_size = dataset_size - train_size

# Split the dataset into train and test
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [16]:
#checking the first batch size
for batch in train_loader:
    inputs, targets = batch
    print(f"Inputs shape: {inputs.shape}")
    print(f"Targets shape: {targets.shape}")
    break  # Inspect only the first batch

Inputs shape: torch.Size([32, 1, 1, 2500])
Targets shape: torch.Size([32])


/tmp/ipykernel_163597/4243377823.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_163597/4243377823.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


In [35]:
len(test_loader)

3

In [26]:
from torcheeg.models import EEGNet

model = EEGNet(chunk_size=2500,
               num_electrodes=1,
               dropout=0.5,
               kernel_1=64,
               kernel_2=16,
               F1=8,
               F2=16,
               D=2,
               num_classes=2)
x, y = next(iter(train_loader))
model(x)

/tmp/ipykernel_163597/4243377823.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_163597/4243377823.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


tensor([[-0.1942, -0.1443],
        [-0.3615,  0.1465],
        [-0.2884,  0.0715],
        [-0.0822, -0.2157],
        [-0.2326, -0.1026],
        [-0.2226, -0.4841],
        [-0.0721, -0.8069],
        [-0.1938, -0.2109],
        [ 0.1731, -0.1928],
        [-0.6245, -0.2708],
        [-0.1099, -0.4050],
        [-0.5204, -0.4799],
        [-0.2436, -0.3875],
        [ 0.4867,  0.3326],
        [ 1.6405,  0.5075],
        [-0.2522,  1.1486],
        [-0.4372, -0.1430],
        [-0.2642, -0.0824],
        [-0.3691,  0.0061],
        [-0.5431,  0.0928],
        [ 0.8441, -0.0132],
        [-0.2296, -0.0322],
        [-0.5753,  0.0324],
        [ 0.7935,  0.7489],
        [ 0.5059,  0.8853],
        [ 0.1258,  0.8852],
        [ 1.1496, -0.1424],
        [ 1.0823,  0.5324],
        [-0.1079,  0.2534],
        [-0.3543, -0.3756],
        [ 0.1077,  0.1820],
        [ 0.2731, -0.1047]], grad_fn=<MmBackward0>)

In [27]:
import torch

import torch.nn as nn
import torch.optim as optim

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.float(), labels.long()  # Ensure proper types

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("Finished Training")

/tmp/ipykernel_163597/4243377823.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_163597/4243377823.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


Epoch [1/100], Loss: 0.6974
Epoch [2/100], Loss: 0.7065
Epoch [3/100], Loss: 0.7144
Epoch [4/100], Loss: 0.6941
Epoch [5/100], Loss: 0.7158
Epoch [6/100], Loss: 0.7029
Epoch [7/100], Loss: 0.7115
Epoch [8/100], Loss: 0.6822
Epoch [9/100], Loss: 0.6966
Epoch [10/100], Loss: 0.6840
Epoch [11/100], Loss: 0.7247
Epoch [12/100], Loss: 0.6969
Epoch [13/100], Loss: 0.6921
Epoch [14/100], Loss: 0.6933
Epoch [15/100], Loss: 0.7072
Epoch [16/100], Loss: 0.7074
Epoch [17/100], Loss: 0.7173
Epoch [18/100], Loss: 0.7091
Epoch [19/100], Loss: 0.7139
Epoch [20/100], Loss: 0.6846
Epoch [21/100], Loss: 0.7000
Epoch [22/100], Loss: 0.6836
Epoch [23/100], Loss: 0.6926
Epoch [24/100], Loss: 0.7104
Epoch [25/100], Loss: 0.7068
Epoch [26/100], Loss: 0.7009
Epoch [27/100], Loss: 0.6981
Epoch [28/100], Loss: 0.7245
Epoch [29/100], Loss: 0.7197
Epoch [30/100], Loss: 0.6992
Epoch [31/100], Loss: 0.6962
Epoch [32/100], Loss: 0.6963
Epoch [33/100], Loss: 0.6980
Epoch [34/100], Loss: 0.6923
Epoch [35/100], Loss: 0

In [34]:
import torch
from torch.nn.functional import softmax

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    label_sum=0

    with torch.no_grad():  # No gradients needed for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.float(), labels.long()  # Ensure proper types
            outputs = model(inputs)  # Forward pass
            predictions = torch.argmax(outputs, dim=1)  # Get predicted class
            correct += (predictions == labels).sum().item()  # Count correct predictions
            total += labels.size(0)  # Total samples
            label_sum += labels.sum().item()

    accuracy = correct / total
    print(label_sum/len(test_loader))
    print(labels)
    return accuracy

# Calculate accuracy
accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.2%}")


12.0
tensor([0, 1, 1, 1, 1, 1, 0, 0])
Test Accuracy: 41.67%


/tmp/ipykernel_163597/4243377823.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_163597/4243377823.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


In [32]:
import torch
from torch.nn.functional import softmax

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No gradients needed for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.float(), labels.long()  # Ensure proper types
            outputs = model(inputs)  # Forward pass
            predictions = torch.argmax(outputs, dim=1)  # Get predicted class
            correct += (predictions == labels).sum().item()  # Count correct predictions
            total += labels.size(0)  # Total samples

    accuracy = correct / total
    return accuracy

# Calculate accuracy
accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.2%}")


Test Accuracy: 41.67%


/tmp/ipykernel_163597/4243377823.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_163597/4243377823.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


In [26]:
model

EEGNet(
  (block1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32), bias=False)
    (1): BatchNorm2d(8, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): Conv2dWithConstraint(8, 16, kernel_size=(2, 1), stride=(1, 1), groups=8, bias=False)
    (3): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (4): ELU(alpha=1.0)
    (5): AvgPool2d(kernel_size=(1, 4), stride=4, padding=0)
    (6): Dropout(p=0.5, inplace=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), padding=(0, 8), groups=16, bias=False)
    (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (2): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (3): ELU(alpha=1.0)
    (4): AvgPool2d(kernel_size=(1, 8), stride=8, padding=0)
    (5): Dropout(p=0.5, inplace=False)
  )
  (lin): Linear(in_features=4992, out_features=2, bias=False)
)

In [27]:
from torcheeg.models import EEGNet

model2 = EEGNet(chunk_size=10000,
               num_electrodes=1,
               dropout=0.5,
               kernel_1=64,
               kernel_2=16,
               F1=8,
               F2=16,
               D=2,
               num_classes=2)
x, y = next(iter(train_loader))
model2

/tmp/ipykernel_157287/4218094284.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).unsqueeze(1)  # Add electrode dimension
/tmp/ipykernel_157287/4218094284.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.stack([torch.tensor(l, dtype=torch.long) for l in label])


EEGNet(
  (block1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32), bias=False)
    (1): BatchNorm2d(8, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): Conv2dWithConstraint(8, 16, kernel_size=(1, 1), stride=(1, 1), groups=8, bias=False)
    (3): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (4): ELU(alpha=1.0)
    (5): AvgPool2d(kernel_size=(1, 4), stride=4, padding=0)
    (6): Dropout(p=0.5, inplace=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), padding=(0, 8), groups=16, bias=False)
    (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (2): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (3): ELU(alpha=1.0)
    (4): AvgPool2d(kernel_size=(1, 8), stride=8, padding=0)
    (5): Dropout(p=0.5, inplace=False)
  )
  (lin): Linear(in_features=4992, out_features=2, bias=False)
)

In [116]:
from torcheeg.models import CCNN
from torcheeg.trainers import ClassifierTrainer
import pytorch_lightning as pl

model = CCNN(num_classes=2, in_channels=1, grid_size=(9, 9))

trainer = ClassifierTrainer(model=model,
                            num_classes=2,
                            lr=1e-4,
                            weight_decay=1e-4,
                            accelerator="gpu")
trainer.fit(train_loader,
            test_loader,
            max_epochs=50,
            default_root_dir=f'scratch/connectome/ahhyun724/DIVER/EcoGpilot/examples_pipeline/model/',
            callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
            enable_progress_bar=True,
            enable_model_summary=True,
            limit_val_batches=0.0)
score = trainer.test(test_loader,
                        enable_progress_bar=True,
                        enable_model_summary=True)[0]
print(f'test accuracy: {score["test_accuracy"]:.4f}')

/scratch/connectome/ahhyun724/.conda/envs/torcheeg/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/connectome/ahhyun724/.conda/envs/torcheeg/l ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | CCNN             | 6.2 M  | train
1 | ce_fn         | CrossEntropyLoss | 0      | train
2 | train_loss    | MeanMetric       | 0      | train
3 | val_loss      | MeanMetric       | 0      | train
4 | test_loss     | MeanMetric       | 0      | train
5 | train_metrics | MetricCollection | 0      | train
6 | val_metrics   | M

Epoch 0:   0%|          | 0/9 [00:00<?, ?it/s] 

RuntimeError: Given groups=1, weight of size [64, 1, 4, 4], expected input[1, 32, 4, 10003] to have 1 channels, but got 32 channels instead

# KFold

In [57]:
from torcheeg.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=42,
           split_path='/scratch/connectome/ahhyun724/DIVER/EcoGpilot/examples_pipeline/split')
            #split_path='./examples_pipeline/split',

In [55]:
for i, (train, val) in enumerate(cv.split(dataset)):
    print(f"Fold {i}")
    print(f"Train: {train}")
    print(f"Val: {val}")

[2025-01-21 06:34:54] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from .torcheeg/model_selection_1737440824410_MMNgS.
[2025-01-21 06:34:54] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.


In [58]:
from torch.utils.data import DataLoader
from torcheeg.models import CCNN

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

for i, (train_dataset, val_dataset) in enumerate(cv.split(dataset)):
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    model = CCNN(num_classes=2, in_channels=1, grid_size=(9, 9))
    print("HERE")
    
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=1e-4,
                                weight_decay=1e-4,
                                accelerator="gpu")
    trainer.fit(train_loader,
                val_loader,
                max_epochs=50,
                default_root_dir=f'/scratch/connectome/ahhyun724/DIVER/EcoGpilot/examples_pipeline/model/{i}',
                callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
                enable_progress_bar=True,
                enable_model_summary=True,
                limit_val_batches=0.0)
    score = trainer.test(val_loader,
                         enable_progress_bar=True,
                         enable_model_summary=True)[0]
    print("HERE")
    print(f'Fold {i} test accuracy: {score["test_accuracy"]:.4f}')

[2025-01-21 06:40:18] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-01-21 06:40:18] INFO (torcheeg/MainThread) 😊 | Please set split_path to /scratch/connectome/ahhyun724/DIVER/EcoGpilot/examples_pipeline/split for the next run, if you want to use the same setting for the experiment.


ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=1.

In [35]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.5.1+cu124.html #torch & python version check and then install


Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 13.8 MB/s eta 0:00:00 0:00:01


In [37]:
from torcheeg.io.meta_info import MetaInfoIO

info_io = MetaInfoIO('/scratch/connectome/ahhyun724/DIVER/EcoG_pilot/metainfo')
key = info_io.write_info({
    'clip_id': 0,
    'baseline_id': 1
})
info = info_io.read_info(key).to_dict()
info

{'clip_id': 0, 'baseline_id': 1}